## Number plate detection

In [1]:
from ultralytics import YOLO
import cv2 as cv
import numpy as np
from sort import *
from utils_ import *

# load models with GPU
model = YOLO('yolov8n.pt').to('cuda')
plate_detector_model = YOLO('../model/license_plate_detector.pt').to('cuda')

mot_tracker = Sort()

# load video
cap = cv.VideoCapture('../data/number_plate_short_hd.mp4')

vehicle_ids = [2, 3, 5, 7]
results = {}

# read frames
frame_nmb = 0
ret = True
while ret:
    ret, frame = cap.read()
    if ret:
        results[frame_nmb] = {}
        # detect vehicles
        vehicles = model(frame)[0]
        vehicles_ = []
        for vehicle in vehicles.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = vehicle
            if int(class_id) in vehicle_ids:
                vehicles_.append([x1, y1, x2, y2, score])

        # track vehicles
        tracking_ids = mot_tracker.update(np.asarray(vehicles_))

        # detect plates
        plates = plate_detector_model(frame)[0]
        print(frame_nmb)

        for plate in plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = plate

            # map plate -> car
            x1car, y1car, x2car, y2car, car_id = map_car(plate, tracking_ids)

            if car_id != -1:
                # crop plate
                cropped_plate = frame[int(y1): int(y2), int(x1): int(x2), :]

                # process plate
                cropped_plate_gray = cv.cvtColor(cropped_plate, cv.COLOR_BGR2GRAY)
                _, plate_thresholded = cv.threshold(cropped_plate_gray, 64, 255, cv.THRESH_BINARY_INV)

                # read the number plate
                result = read_license_plate(plate_thresholded)
                if result is not None:
                    license_number, license_number_score = result
                else:
                    license_number, license_number_score = "-1", "-1"

                if license_number != -1 and not None:
                    results[frame_nmb][car_id] = {'car': {'bbox': [x1car, y1car, x2car, y2car]},
                                                  'plate': {'bbox': [x1, y1, x2, y2],
                                                            'text': license_number,
                                                            'bbox_score': score,
                                                            'text_score': license_number_score}}
        frame_nmb += 1
    else:
        break

# write results
write_csv(results, '../results/test.csv')

Using CPU. Note: This module is much faster with a GPU.



0: 384x640 21 cars, 1 bus, 2 trucks, 23.8ms
Speed: 5.4ms preprocess, 23.8ms inference, 35.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 14.2ms
Speed: 7.0ms preprocess, 14.2ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)
0

0: 384x640 22 cars, 1 bus, 2 trucks, 20.9ms
Speed: 0.0ms preprocess, 20.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 16.0ms
Speed: 8.0ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
1

0: 384x640 22 cars, 1 bus, 2 trucks, 21.3ms
Speed: 0.0ms preprocess, 21.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 15.5ms
Speed: 5.3ms preprocess, 15.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
2

0: 384x640 23 cars, 1 bus, 1 truck, 12.2ms
Speed: 0.0ms preprocess, 12.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_pl